In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time
import random

In [2]:
companies = [
    'www.booking.com',
    'www.cheaptickets.nl',
    'www.bol.com',
    'www.coolblue.nl',
    'www.wehkamp.nl',
    'www.zalando.nl',
    'www.mediamarkt.nl',
    'www.debijenkorf.nl',
    
]

In [22]:
def get_no_pages(company_url):
    
    resp = requests.get(f'https://www.trustpilot.com/review/{company_url}')
    
    ### Select relevant HTML elements of webpage
    soup = BeautifulSoup(resp.text, 'html.parser')
    reviews = soup.find_all('article', class_='review')

    total_reviews = json.loads(soup.find('script', attrs={'data-initial-state': 'review-list'}).string)['totalNumberOfReviews']
    pages = round(total_reviews / 20)
    
    return pages
    

In [16]:
pages, soup = get_no_pages('www.booking.com')

In [ ]:
review.find('div', attrs={'class': 'consumer-information__name'}):
                tmp.append(review.find('div', attrs={'class': 'consumer-information__name'}).text.strip())


In [19]:
soup.find('a', attrs={'data-page-number': 'next-page'})['href']

'/review/www.booking.com?b=MTYxODE1OTcwODAwMHw2MDczMjg1Y2Y4NWQ3NTA4NzA1NWYzMTc'

In [44]:
def scrape_info(max_pages=5):
    
    ### Get url to make request to
    company_url = input("Enter url-adress of company of interest")
    
    pages = get_no_pages(company_url)
    
    request_url = f'https://www.trustpilot.com/review/{company_url}'
    
    records = []
    
    for page in range(1, pages+1):
        if page > max_pages:
            break
        
#         request_url = f'https://www.trustpilot.com/review/{company_url}?languages=nl&page={page}'
#         request_url = f'https://www.trustpilot.com/review/{company_url}?page={page}'

        
        time.sleep(random.uniform(2, 6))
        
        ### Make the request
        resp = requests.get(request_url)
        print(f'response: {resp}')

        ### Select relevant HTML elements of webpage
        soup = BeautifulSoup(resp.text, 'html.parser')
        reviews = soup.find_all('article', class_='review')
        
        next_url = soup.find('a', attrs={'data-page-number': 'next-page'})['href']
        request_url = f'https://www.trustpilot.com{next_url}'

        ### Extract the desired information
        for review in reviews:
            tmp = []

            # name
            if review.find('div', attrs={'class': 'consumer-information__name'}):
                tmp.append(review.find('div', attrs={'class': 'consumer-information__name'}).text.strip())

            # number of reviews
            if review.find('div', attrs={'class': 'consumer-information__review-count'}):
                tmp.append(review.find('div', attrs={'class': 'consumer-information__review-count'}).text.strip())

            # location
            if review.find('div', attrs={'class': 'consumer-information__location'}):
                tmp.append(review.find('div', attrs={'class': 'consumer-information__location'}).text.strip())

            # rating
            if review.find("div", {"class":"star-rating star-rating--medium"}):
                tmp.append(review.find("div", {"class":"star-rating star-rating--medium"}).find('img').get('alt'))

            # review header
            tmp.append(json.loads(review.find('script').string)['reviewHeader'])

            # review text
            tmp.append(json.loads(review.find('script').string)['reviewBody'])

            # time published
            tmp.append(json.loads(review.find('script', attrs={'data-initial-state': 'review-dates'}).string)['publishedDate'])

            # if company reply
            if review.find('div', attrs={'class': 'review__company-reply'}):
                # responded
                tmp.append(1)

                # time published
                tmp.append(json.loads(review.find('script', attrs={'data-initial-state': 'review-reply-dates'}).string)['publishedDate'])

                # response
                tmp.append(review.find('div', attrs={'class': 'brand-company-reply__content'}).text.strip()) 
            else:
                # not responded
                tmp.append(0)
                tmp.append(None)
                tmp.append(None)
                
            records.append(tmp)

        
    ### Build pandas dataframe with review data
    
    # column names
    labels = ['name', 'nb_reviews', 'location', 'rating', 'header', 'content', 'date', 'responded', 'response_date', 'response']
    
    # create dataframe
    df = pd.DataFrame.from_records(records, columns=labels)
    
    ### Clean data
    
    df['nb_reviews'] = df['nb_reviews'].apply(lambda x: int(x.split()[0]))
    df['rating'] = df['rating'].apply(lambda x: int(x.split()[0]))
    df['date'] = pd.to_datetime(df['date'])
    df['response_date'] = pd.to_datetime(df['response_date'])
    df['response'] = df['response'].fillna('-')
    
    return df

In [5]:
companies

['www.booking.com',
 'www.cheaptickets.nl',
 'www.bol.com',
 'www.coolblue.nl',
 'www.wehkamp.nl',
 'www.zalando.nl',
 'www.mediamarkt.nl',
 'www.debijenkorf.nl']

In [45]:
# Scrape reviews
booking = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.booking.com
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [47]:
cheaptickets = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.cheaptickets.nl
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [48]:
bol = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.bol.com
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [49]:
coolblue = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.coolblue.nl
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [50]:
wehkamp = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.wehkamp.nl
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [51]:
zalando = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.zalando.nl
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [52]:
mediamarkt = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.mediamarkt.nl
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [53]:
debijenkorf = scrape_info(max_pages=5)

Enter url-adress of company of interestwww.debijenkorf.nl
response: <Response [200]>
response: <Response [200]>
response: <Response [200]>


In [54]:
# Add company column to dataframe
booking['company'] = 'booking.com' 

cheaptickets['company'] = 'cheaptickets.nl' 

bol['company'] = 'bol.com' 

coolblue['company'] = 'coolblue.nl' 

wehkamp['company'] = 'wehkamp.nl' 

zalando['company'] = 'zalando.nl' 

mediamarkt['company'] = 'mediamarkt.nl' 

debijenkorf['company'] = 'debijenkorf.nl' 

In [55]:
combined = pd.concat([booking, cheaptickets, bol, 
                      coolblue, wehkamp, zalando, 
                      mediamarkt, debijenkorf])
combined.head()

,name,nb_reviews,location,rating,header,content,date,responded,response_date,response,company
0,Sherry Angel,1,US,1,Bad Bad Bad Company,"I booked a villa through this company, found a...",2021-04-20 05:57:52+00:00,0,NaT,-,booking.com
1,David,1,US,1,I cancelled my reservation and booking.com nev...,"If I could give them a negative 5 star review,...",2021-04-20 01:40:14+00:00,0,NaT,-,booking.com
2,Alexander Santos,1,US,1,Terrible experience with this website,Terrible experience with this website. I made ...,2021-04-19 19:26:11+00:00,0,NaT,-,booking.com
3,G May,27,GB,1,BUY ANYWHERE ELSE,Booked a holiday...took my money. Then a few ...,2021-04-18 21:19:41+00:00,0,NaT,-,booking.com
4,Lee Mac,3,GB,1,Avoid Booking.com,I would just like to take the time to warn eve...,2021-04-18 19:11:58+00:00,0,NaT,-,booking.com


In [63]:
combined.to_csv('combined_reviews_eng.csv')